In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import *
from torch.utils.data import SubsetRandomSampler, Dataset
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import splitfolders

torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


In [ ]:
absolute_path = os.path.abspath("")
relative_path = 'data\images'
PATH = '.\data\images'

# splitfolders.ratio('E:\dataset2-master\dataset2-master\images\WBC_DATA', output='E:\dataset2-master\dataset2-master\images', seed=42, ratio=(0.8, 0.1, 0.1), move=False)

# pre-normalized transformation
transform1 = transforms.Compose([transforms.Resize((120,120))
                                , transforms.ToTensor()
                                ])

train_data = datasets.ImageFolder(PATH + '\TRAIN', transform=transform1)
val_data = datasets.ImageFolder(PATH + '\VAL', transform=transform1)
test_data = datasets.ImageFolder(PATH + '\TEST', transform=transform1)
simple_test_data = datasets.ImageFolder(PATH + '\TEST_SIMPLE', transform=transform1)

# pre-normalized dataloaders
original_dataloaders = {
    "train": torch.utils.data.DataLoader(
        train_data
    ),
    "validation": torch.utils.data.DataLoader(
        val_data, shuffle=False
    ),
    "test": torch.utils.data.DataLoader(
        test_data, batch_size = 128
    ),
    "simple test": torch.utils.data.DataLoader(
        simple_test_data
    )
}

def calculate_mean_and_std(loader):
    sum, squared_sum, num_batches = 0, 0, 0
    for data,_ in loader:
        sum += torch.mean(data, dim=[0, 2, 3])
        squared_sum += torch.mean(data**2, dim=[0, 2, 3])
        num_batches += 1

    mean = sum / num_batches
    std = (squared_sum / num_batches - mean**2)**0.5

    return mean, std

mean,std = calculate_mean_and_std(original_dataloaders.get('train'))
print(mean)
print(std)

# normalized transformation
norm_transform = transforms.Compose([transforms.Resize((120,120))
                                , transforms.ToTensor()
                                , transforms.Normalize((mean), (std))
                                ])

train_data = datasets.ImageFolder(PATH + '\TRAIN', transform=norm_transform)
val_data = datasets.ImageFolder(PATH + '\VAL', transform=norm_transform)
test_data = datasets.ImageFolder(PATH + '\TEST', transform=norm_transform)
simple_test_data = datasets.ImageFolder(PATH + '\TEST_SIMPLE', transform=norm_transform)

# normalized dataloaders
dataloaders = {
    "train": torch.utils.data.DataLoader(
        train_data, batch_size = 128, shuffle=True, num_workers=4, pin_memory=True
    ),
    "validation": torch.utils.data.DataLoader(
        val_data, batch_size = 128, shuffle=False, num_workers=4, pin_memory=True
    ),
    "test": torch.utils.data.DataLoader(
        test_data, batch_size = 128
    ),
    "simple test": torch.utils.data.DataLoader(
        simple_test_data
    )
}

train_count = len(train_data)
validation_count = len(val_data)



In [ ]:
# simple cnn
simple_cnn = nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=6, kernel_size=3, stride=1)
    , nn.ReLU()
    , nn.Flatten()
    , nn.Linear(in_features=27144, out_features=500)
    , nn.Linear(in_features=500, out_features=4)
    , nn.LogSoftmax(dim=1)
)

# dropout nn
network1 = nn.Sequential(

    nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
    , nn.Dropout(0.2)
    , nn.ReLU()
    , nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, stride=1, padding=1)
    , nn.Dropout(0.2)
    , nn.ReLU()
    , nn.MaxPool2d(kernel_size=2, stride=2)

    , nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
    , nn.Dropout(0.2)
    , nn.ReLU()
    , nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1)
    , nn.Dropout(0.2)
    , nn.ReLU()
    , nn.MaxPool2d(kernel_size=2, stride=2)

    , nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
    , nn.Dropout(0.2)
    , nn.ReLU()
    , nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
    , nn.Dropout(0.2)
    , nn.ReLU()
    , nn.MaxPool2d(kernel_size=2, stride=2)

    , nn.Flatten()
    , nn.Linear(in_features=19200, out_features=3000)
    , nn.Dropout(0.2)
    , nn.Linear(in_features=3000, out_features=600)
    , nn.Dropout(0.2)
    , nn.Linear(in_features=600, out_features=32)
    , nn.Dropout(0.2)
    , nn.Linear(in_features=32, out_features=4)
)

# delayed dropout
network2 = nn.Sequential(

    nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
    , nn.ReLU()
    , nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, stride=1, padding=1)
    , nn.ReLU()
    , nn.MaxPool2d(kernel_size=2, stride=2)

    , nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
    , nn.ReLU()
    , nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1)
    , nn.ReLU()
    , nn.MaxPool2d(kernel_size=2, stride=2)

    , nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
    , nn.ReLU()
    , nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
    , nn.ReLU()
    , nn.MaxPool2d(kernel_size=2, stride=2)

    , nn.Flatten()
    , nn.Linear(in_features=19200, out_features=3000)
    , nn.Dropout(0.2)
    , nn.Linear(in_features=3000, out_features=600)
    , nn.Dropout(0.2)
    , nn.Linear(in_features=600, out_features=32)
    , nn.Dropout(0.2)
    , nn.Linear(in_features=32, out_features=4)
)

# batch normalization cnn 2
network3 = nn.Sequential(

    nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
    , nn.BatchNorm2d(32)
    , nn.ReLU()
    , nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1)
    , nn.BatchNorm2d(32)
    , nn.ReLU()
    , nn.MaxPool2d(kernel_size=2, stride=2)

    , nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
    , nn.BatchNorm2d(64)
    , nn.ReLU()
    , nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
    , nn.BatchNorm2d(64)
    , nn.ReLU()
    , nn.MaxPool2d(kernel_size=2, stride=2)

    #, nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
    #, nn.ReLU()
    #, nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
    #, nn.ReLU()
    #, nn.MaxPool2d(kernel_size=2, stride=2)

    , nn.Flatten()
    , nn.Linear(in_features=19200, out_features=3000)
    , nn.Dropout(0.5)
    , nn.Linear(in_features=3000, out_features=600)
    , nn.Dropout(0.5)
    , nn.Linear(in_features=600, out_features=32)
    , nn.Dropout(0.5)
    , nn.Linear(in_features=32, out_features=4)
    , nn.LogSoftmax(dim=1)
)

# larger kernel nn
network4 = nn.Sequential(

    nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=4)
    , nn.ReLU()
    , nn.Conv2d(in_channels=16, out_channels=16, kernel_size=5, stride=1, padding=4)
    , nn.ReLU()
    , nn.MaxPool2d(kernel_size=2, stride=2)

    , nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=4)
    , nn.ReLU()
    , nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5, stride=1, padding=4)
    , nn.ReLU()
    , nn.MaxPool2d(kernel_size=2, stride=2)

    , nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=4)
    , nn.ReLU()
    , nn.Conv2d(in_channels=64, out_channels=64, kernel_size=5, stride=1, padding=4)
    , nn.ReLU()
    , nn.MaxPool2d(kernel_size=2, stride=2)

    , nn.Flatten()
    , nn.Linear(in_features=38016, out_features=3000)
    , nn.Dropout(0.2)
    , nn.ReLU()
    , nn.Linear(in_features=3000, out_features=600)
    , nn.Dropout(0.2)
    , nn.ReLU()
    , nn.Linear(in_features=600, out_features=32)
    , nn.Dropout(0.2)
    , nn.ReLU()
    , nn.Linear(in_features=32, out_features=4)
)


In [1]:

class Run_Network:

    def __init__(self, name: str, network, learning_rate, epochs):
        self.name = name
        self.network = network
        self.learning_rate = learning_rate

        self.epochs = epochs
        self.loss_values = []
        self.valid_loss_values = []
        self.train_acc = []
        self.val_acc = []

    def train_network(self):
        self.network.to(device)

        #define optimizer and scheduler and cost function
        optimizer = torch.optim.Adam(self.network.parameters(), lr=self.learning_rate)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=20)
        criterion = nn.CrossEntropyLoss()

        for epoch in range(self.epochs):  # loop over the dataset multiple times
            self.network.train()

            tr_correct = 0
            tr_total = 0
            running_loss = 0.0
            
            for i, data in enumerate(dataloaders.get('train'), 0):
                # get the inputs; data is a list of [inputs, labels]
                inputs, labels = data[0].to(device), data[1].to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward + backward + optimize
                outputs = self.network(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                #training loss and accuracy
                running_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs.data, 1)
                tr_total += labels.size(0)
                tr_correct += (predicted == labels).sum().item()

            scheduler.step()    
            
            self.train_acc.append(100 * tr_correct / tr_total)   
            self.loss_values.append(running_loss / train_count)
            
            rounded_accuracy = format(100 * tr_correct / tr_total, '.2f')
            self.train_acc.append(100 * tr_correct / tr_total)   
            self.loss_values.append(running_loss / train_count)
            
            print(f'VAL LOSS: {loss} -- VAL ACCURACY: {rounded_accuracy} %')
                
            correct = 0
            total = 0
            running_valid_loss = 0.0

            # since we're not training, we don't need to calculate the gradients for our outputs
            with torch.no_grad():
                self.network.eval()

                for data in dataloaders.get('validation'):
                    images, labels = data[0].to(device), data[1].to(device)
                    # calculate outputs by running images through the network
                    outputs = self.network(images)
                    # the class with the highest energy is what we choose as prediction
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
                    valid_loss = criterion(outputs, labels)
                    running_valid_loss += valid_loss.item() * images.size(0)
            
            rounded_accuracy = format(100 * correct / total, '.2f')        
            self.val_acc.append(rounded_accuracy)
            self.valid_loss_values.append(running_valid_loss / validation_count)
        
            print(f'VAL LOSS: {valid_loss} -- VAL ACCURACY: {rounded_accuracy} % -- EPOCH: {epoch}')

    def test_network(self):
        self.network.to(device)
        self.network.eval()
        
        criterion = nn.CrossEntropyLoss()
        correct = 0
        total = 0
        running_valid_loss = 0.0

        with torch.no_grad():
            for data in dataloaders.get('simple test'):
                images, labels = data[0].to(device), data[1].to(device)
                # calculate outputs by running images through the network
                outputs = self.network(images)
                # the class with the highest energy is what we choose as prediction
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                valid_loss = criterion(outputs, labels)
                running_valid_loss += valid_loss.item() * images.size(0)
            
            rounded_accuracy = format(100 * correct / total, '.2f')
            self.val_acc.append(rounded_accuracy)
            self.valid_loss_values.append(running_valid_loss / validation_count)
        
        print(f'TEST LOSS: {valid_loss} -- TEST ACCURACY: {rounded_accuracy} %')

    def plot_stats(self, save_plot=False):
        plt.plot(self.loss_values, label='train loss')
        plt.plot(self.valid_loss_values, label='valid loss')
        plt.legend()
        plt.title(f'{self.name} Loss')
        if save_plot==True:
            plt.savefig(f'.\model_performance_data\{self.name}_Loss.png')
        plt.show()

        plt.plot(self.train_acc, label='train acc')
        plt.plot(self.val_acc, label='valid acc')
        plt.legend()
        plt.title(f'{self.name} Accuracy')
        if save_plot==True:
            plt.savefig(f'.\model_performance_data\{self.name}_Acc.png')
        plt.show()

    def get_confusion_matrix(self, dataloader='validation', save_plot=False):
        self.network.eval()
        y_pred = []
        y_true = []
        
        with torch.no_grad():
            for data in dataloaders.get(dataloader):
                images, labels = data[0].to(device), data[1].to(device)
                # calculate outputs by running images through the network
                output = self.network(images)
                
                output = (torch.max(torch.exp(output), 1)[1]).data.cpu().numpy()
                y_pred.extend(output)
                
                labels = labels.data.cpu().numpy()
                y_true.extend(labels)
                
                classes = ('EOSINOPHIL','LYMPHOCYTE','MONOCYTE','NEUTROPHIL')
            
            cfm = confusion_matrix(y_true, y_pred)
            df_cfm = pd.DataFrame(cfm/sum(cfm), index = [i for i in classes], columns = [i for i in classes])

            cm = sns.heatmap(df_cfm, annot = True, cmap = 'Greens')
            plt.title(self.name)
            if save_plot==True:
                plt.savefig(f'.\model_performance_data\{self.name}_CM.png')


    def clear_loss_and_accuracy(self):
        self.loss_values.clear()
        self.valid_loss_values.clear()
        self.train_acc.clear()
        self.val_acc.clear()
    
    def save_model(self):
        torch.save(self.network.state_dict(), f'.\model_states\{self.name}.pt')

In [ ]:

best_net = Run_Network(
    name="network1"
    , network=network1
    , learning_rate=0.0001
    , epochs=100
)

dropout_and_batch_norm = Run_Network(
    name="dropout + batch norm"
    , network=network2
    , learning_rate=0.0001
    , epochs=100
)

large_first_kernel = Run_Network(
    name="5x5 Kernel"
    , network=network4
    , learning_rate=0.0001
    , epochs=100
)

large_first_kernel.train_network()
large_first_kernel.plot_stats()
large_first_kernel.get_confusion_matrix()
#large_first_kernel.save_model()
large_first_kernel.test_network()
large_first_kernel.get_confusion_matrix(dataloader='test')


print(df_cfm)
print(cfm)
print(sum(cfm))
print(cfm/sum(cfm) * 100)
print(sum(cfm/sum(cfm) * 100))